In [1]:
import torch
import os
from typing import Tuple
import pandas as pd
import json

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
cache_path = "/Users/tonykuo/Downloads/harness_cache_20240428"

In [3]:
# walk through the cache path
all_cache_paths = []
for root, dirs, files in os.walk(cache_path):
    for file in files:
        if file.endswith(".pt"):
            all_cache_paths.append(os.path.join(root, file))

In [4]:
# all_cache_paths

In [5]:
tasks = [
    "mmlu_flan_n_shot_generative",
    # "truthfulqa_gen",
    # "gsm8k",
    # "bbh_fewshot",
    # "triviaqa",
    # "nq_open"
]

ranker_name = [
    "llm_blender",
    "oassterm",
    "ultrarm"
]

analize_data = [
    {
        "types": "single_model", 
        "display_name": "openchat/openchat_3.5",
        "model_name":["openchat/openchat_3.5"]
    },
    {
        "types": "single_model", 
        "display_name": "NousResearch/Nous-Hermes-2-SOLAR-10.7B",
        "model_name":["NousResearch/Nous-Hermes-2-SOLAR-10.7B"]
    },
    {
        "types": "single_model", 
        "display_name": "FuseAI/OpenChat-3.5-7B-Solar",
        "model_name":["FuseAI/OpenChat-3.5-7B-Solar"]
    },
    # {
    #     "types": "ranker", 
    #     "display_name": "llm_blender",
    #     "model_name":["openchat/openchat_3.5", "NousResearch/Nous-Hermes-2-SOLAR-10.7B"],
    #     "ranker_name": "llm_blender"
    # },
    {
        "types": "ranker", 
        "display_name": "oassterm",
        "model_name": ["openchat/openchat_3.5", "NousResearch/Nous-Hermes-2-SOLAR-10.7B"],
        "ranker_name": "oassterm"
    },
    # {
    #     "types": "ranker", 
    #     "display_name": "ultrarm",
    #     "model_name":["openchat/openchat_3.5", "NousResearch/Nous-Hermes-2-SOLAR-10.7B"],
    #     "ranker_name": "ultrarm"
    # }
]

In [6]:

def custom_split_v4(s):
    skip_str = [
        "mmlu_flan_n_shot_generative",
        "truthfulqa_gen",
        "gsm8k",
        "bbh_fewshot",
        "triviaqa",
        "nq_open",
        "llm_blender",
        "openchat-openchat_3.5"
    ]


    parts = s.split('_')
    result = []
    i = 0

    while i < len(parts):
        current = parts[i]
        match_found = False

        # Check if the current part can form a task with the next few parts
        for j in range(1, len(parts) - i):
            combined = '_'.join(parts[i:i+j+1])
            if combined in skip_str:
                result.append(combined)
                i += j  # Move the index to the end of the combined part
                match_found = True
                break

        if not match_found:
            result.append(current)
        i += 1

    return result


def parse_cache_paths(all_cache_paths) -> pd.DataFrame:
    accept_tasks = [
        "mmlu_flan_n_shot_generative",
        "truthfulqa_gen",
        "gsm8k",
        "bbh_fewshot",
        "triviaqa",
        "nq_open"
    ]
    pd_parse_data = [custom_split_v4(os.path.basename(path)[:-3]) for path in all_cache_paths]

    pd_ranker_names = []
    for d in pd_parse_data:
        if d[0] in ranker_name:
            pd_ranker_names.append(d[0])
            d.pop(0)
        else:
            pd_ranker_names.append(None)

    pd_task_names = []
    for d in pd_parse_data:
        if d[0] in accept_tasks:
            # print(d)
            pd_task_names.append(d[0])
            d.pop(0)
        else:
            raise ValueError("Task name not found")
        

    return pd.DataFrame.from_dict(
        {
            "cache_path": all_cache_paths,
            "ranker_names": pd_ranker_names,
            "task_names": pd_task_names,
            "models": pd_parse_data
        }
    )



In [7]:
all_cache_paths_df = parse_cache_paths(all_cache_paths)

In [8]:
all_cache_paths_df

,cache_path,ranker_names,task_names,models
0,/Users/tonykuo/Downloads/harness_cache_2024042...,oassterm,triviaqa,"[meta-llama-Meta-Llama-3-8B-Instruct, mistrala..."
1,/Users/tonykuo/Downloads/harness_cache_2024042...,ultrarm,truthfulqa_gen,"[NousResearch-Nous-Hermes-2-SOLAR-10.7B, openc..."
2,/Users/tonykuo/Downloads/harness_cache_2024042...,None,mmlu_flan_n_shot_generative,[FuseAI-OpenChat-3.5-7B-Solar]
3,/Users/tonykuo/Downloads/harness_cache_2024042...,oassterm,mmlu_flan_n_shot_generative,"[meta-llama-Meta-Llama-3-8B-Instruct, mistrala..."
4,/Users/tonykuo/Downloads/harness_cache_2024042...,ultrarm,nq_open,"[NousResearch-Nous-Hermes-2-SOLAR-10.7B, openc..."
...,...,...,...,...
91,/Users/tonykuo/Downloads/harness_cache_2024042...,None,nq_open,[openchat-openchat_3.5]
92,/Users/tonykuo/Downloads/harness_cache_2024042...,None,gsm8k,[openchat-openchat_3.5]
93,/Users/tonykuo/Downloads/harness_cache_2024042...,llm_blender,nq_open,"[openchat-openchat_3.5, NousResearch-Nous-Herm..."
94,/Users/tonykuo/Downloads/harness_cache_2024042...,None,gsm8k,[NousResearch-Nous-Hermes-2-Mixtral-8x7B-DPO]


In [9]:
def parse_filter(cache_df: pd.DataFrame, task:str, models:list[str], ranker_name:str|None= None) -> str:
    cache_df = cache_df.copy()
    if ranker_name is not None:
        assert  len(models) > 1, "ranker_name should have more than 1 model_name"
    
    
    if ranker_name is None:
        cache_df = cache_df[cache_df["ranker_names"].isna()]
    else:
        cache_df = cache_df[cache_df["ranker_names"] == ranker_name]

    cache_df = cache_df[cache_df["task_names"] == task]

    # all models in cache_df["models"], regardless of the order
    cache_df["models"] = cache_df["models"].apply(sorted)
    cache_df["models"] = cache_df["models"].apply(lambda x: tuple(x))
    cache_df = cache_df[cache_df["models"] == tuple(sorted(models))]

    # print(cache_df.shape)
    assert len(cache_df) == 1, "There should be only one cache path"
    
    return cache_df.cache_path.to_list()[0]



In [10]:
def load_cache(cache_path: str, pop_first: bool = True) -> dict:
    assert os.path.exists(cache_path), f"Cache path not found, {cache_path}"
    cache = torch.load(cache_path)
    if pop_first:
        _ = cache.pop(list(cache.keys())[0])
    return cache


def calculate_from_cache_path(cache_path: str, ranker: bool, pop_first: bool = True) -> Tuple[float, int]:    
    cache = load_cache(cache_path, pop_first=pop_first)
    
    total_latency = 0
    total_generated_token = 0
    for _, data in cache.items():
        if ranker:
            # print(data)
            total_latency += data['latency']
        else:
            total_latency += data['latency']['generate_latency']
            total_generated_token += data['latency']['num_generate_tokens']
        
    return total_latency, total_generated_token

In [11]:
def single_model_flow(data:dict, task:str):
    target_models = [target_model.replace("/", "-") for target_model in data["model_name"]]

    cache_path = parse_filter(
        cache_df=all_cache_paths_df,
        task=task,
        models=target_models,
        ranker_name=data.get("ranker_name", None)
    )
    total_latency, total_generated_token = calculate_from_cache_path(
        cache_path=cache_path,
        ranker=data["types"] == "ranker",
        pop_first=True
    )
    return total_latency, total_generated_token

def ranker_flow(data:dict, task:str):
    target_models = [target_model.replace("/", "-") for target_model in data["model_name"]]

    # init dict
    cache_path = parse_filter(
        cache_df=all_cache_paths_df,
        task=task,
        models=target_models,
        ranker_name=data.get("ranker_name", None)
    )

    ranker_result = load_cache(cache_path, pop_first=True)  
    """
    {"...question...":
        {
            "candidate_dict": {
                "openchat/openchat_3.5": {
                    "text": "(A)",
                    "latency": 0.05516314506530762
                },
                "NousResearch/Nous-Hermes-2-SOLAR-10.7B": {
                    "text": "(C)",
                    "latency": 0.05516314506530762
                }
            },
            "final_candidate_model": "NousResearch/Nous-Hermes-2-SOLAR-10.7B",
            "final_rank_result": "(C)"
        },
        ...
    }
    """
    for target_model in data["model_name"]:
        tm = target_model.replace("/", "-")
        cache_path = parse_filter(
            cache_df=all_cache_paths_df,
            task=task,
            models=[tm],
            ranker_name=None
        )
        # print(cache_path)
        target_model_result = load_cache(cache_path, pop_first=True)
        """
        {
            "...quesion...": {
                "response": "Bobby Scott and Bob Russell wrote the lyrics to \"He Ain't Heavy",
                "latency": {
                    "generate_latency": 0.4029510021209717,
                    "num_generate_tokens": 4
                }
            }
        }
        """
        for q in target_model_result:
            ranker_result[q]['candidate_dict'][target_model]['latency'] += target_model_result[q]['latency']['generate_latency']
            ranker_result[q]['candidate_dict'][target_model]['num_generate_tokens'] = target_model_result[q]['latency']['num_generate_tokens']

    # final result
    """
    {"...question...":
        {
            "candidate_dict": {
                "openchat/openchat_3.5": {
                    "text": "(A)",
                    "latency": 0.05516314506530762,
                    "num_generate_tokens": 4
                },
                "NousResearch/Nous-Hermes-2-SOLAR-10.7B": {
                    "text": "(C)",
                    "latency": 0.05516314506530762,
                    "num_generate_tokens": 4
                }
            },
            "final_candidate_model": "NousResearch/Nous-Hermes-2-SOLAR-10.7B",
            "final_rank_result": "(C)"
        },
        ...
    }
    """
    total_latency = 0
    total_generated_token = 0
    for q in ranker_result:
        each_latency = []

        # get max latency
        for _, v in ranker_result[q]['candidate_dict'].items():
            each_latency.append(v['latency'])
        total_latency += max(each_latency)

        # get total generated token by final selected model
        final_selected_model = ranker_result[q]['final_candidate_model']
        total_generated_token += ranker_result[q]['candidate_dict'][final_selected_model]['num_generate_tokens']

    return total_latency, total_generated_token

In [12]:
def calculate_result_table(analize_data:dict):
    display_name = [data['display_name'] for data in analize_data]
    display_name.insert(0, "task")
    _df = pd.DataFrame(columns=display_name)

    for task in tasks:
        _result = {"task": task}
        for data in analize_data:
            if data["types"] == "single_model":
                total_latency, total_generated_token = single_model_flow(data, task)
            elif data["types"] == "ranker":
                #print(f"{data=}")
                total_latency, total_generated_token = ranker_flow(data, task)
            else:
                raise ValueError("types should be single_model or ranker")
            
            _result[data["display_name"]] = f"{round(total_latency/total_generated_token, 3)} s/token"

        # print(_result)
        
        # add a row to the dataframe
        _df = _df._append(_result, ignore_index=True)


    return _df



In [13]:
result_tb = calculate_result_table(analize_data)
result_tb

,task,openchat/openchat_3.5,NousResearch/Nous-Hermes-2-SOLAR-10.7B,FuseAI/OpenChat-3.5-7B-Solar,oassterm
0,mmlu_flan_n_shot_generative,31.951 tok/s,17.876 tok/s,30.381 tok/s,18.634 tok/s
